In [ ]:
from everest.ptolemaic import ODEModelTemplate

class MyModel(ODEModelTemplate):

    a: int
    b: float
    c: bool

    ...

everest.window.Window(MyModel[asdf;asdflj, init:final, myanalysers])

In [ ]:
for param_combination in combinations_of([aasdfasdf], [123123123]):
    mycase = MyModel(*paramasdfalksdjnf)

In [ ]:
mycombos = MyModel[allpositiveintegers][randomsample]

In [ ]:
MyModel[myparameters, myinitialcondition : myfinalcondition]

In [ ]:
Array('asdfasdfasjdf')

In [ ]:
myarr ** 2 -> futures

In [ ]:
mycombos.calculate()